该教程来自AWS案例, 点击[这里](https://github.com/aws/amazon-sagemaker-examples/blob/main/autopilot/autopilot_time_series.ipynb)查看原案例代码.

In [7]:
# 安装依赖
!pip install --upgrade boto3 --quiet
!pip install --upgrade sagemaker --quiet


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [19]:
# 运行时参数
import os

os.environ["AWS_PROFILE"] = "sandbox"

In [27]:
# Setup
import sagemaker
import boto3
from sagemaker import get_execution_role
from time import gmtime, strftime, sleep
import datetime

region = boto3.Session().region_name
session = sagemaker.Session()

# Modify the following default_bucket to use a bucket of your choosing
# bucket = session.default_bucket()
bucket = 'sandbox-sagemaker' 
prefix = 'autopilot_time_series'
role = "arn:aws-cn:iam::278103880173:role/sandbox-SageMakerExecutorRole"

print("basic info: ", {"region": region, "bucket":bucket, "prefix": prefix, "role": role})
# This is the client we will use to interact with SageMaker Autopilot
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

basic info:  {'region': 'cn-northwest-1', 'bucket': 'sandbox-sagemaker', 'prefix': 'autopilot_time_series', 'role': 'arn:aws-cn:iam::278103880173:role/sandbox-SageMakerExecutorRole'}


准备预测数据, 预测数据是一个合成的食品需求清单, 包含以下几列:
- product_code (required: ItemIdentifierAttributeName)
- product_category (static, categorical feature describing product_code)
- product_subcategory (static, categorical feature describing product_code)
- location_code (GroupingAttributeNames column to get predictions at product_code + location_code)
- scaled_price (covariate)
- promotion_email (covariate)
- promotion_homepage (covariate)
- timestamp (required, TimestampAttributeName)
- unit_sales (required: TargetAttributeName)

[测试数据](https://amazon-forecast-samples.s3.amazonaws.com/automation_solution/demo-nyctaxi/nyctaxi_weather_auto.csv)已经下载到本地*data*目录, 但是在实际测试发现AutoMLJobV2不支持, 而AutoMLJob又不支持配置缺失数据, 所以用下面的脚本增加缺失数据默认值

In [64]:
import pandas as pd
import json

# Load the dataset
df = pd.read_csv('data/real-time-payload.csv')

# Check for hidden characters or extra spaces in 'unit_sales' column
df['unit_sales'] = df['unit_sales'].apply(lambda x: str(x).strip() if pd.notnull(x) else x)

# Convert 'unit_sales' column to numeric, forcing errors to NaN
df['unit_sales'] = pd.to_numeric(df['unit_sales'], errors='coerce')

# Replace null values with 0
df['unit_sales'].fillna(0, inplace=True)

# Verify there are no missing values left
missing_values = df['unit_sales'].isnull().sum()
print(f'Total missing values in unit_sales after filling: {missing_values}')

# Save the cleaned dataset
df.to_csv('data/default-real-time-payload.csv', index=False)

Total missing values in unit_sales after filling: 0


/var/folders/yt/3470fws56wqck_qkbkd3jyx80000gn/T/ipykernel_78078/1868299028.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['unit_sales'].fillna(0, inplace=True)


In [47]:
s3 = boto3.client('s3')
s3.upload_file('data/default-synthetic-food-demand.csv', bucket, f'{prefix}/data/synthetic-food-demand.csv')

In [48]:
# 模型训练
timestamp_suffix = strftime("%Y%m%d-%H%M%S", gmtime())
auto_ml_job_name = "ts-" + timestamp_suffix
print("AutoMLJobName: " + auto_ml_job_name)

AutoMLJobName: ts-20240807-094452


In [53]:
input_data_config = [
    {  
      'ChannelType': 'training',
      'ContentType': 'text/csv;header=present',
      'CompressionType': 'None',
      'DataSource': {
        'S3DataSource': {
          'S3DataType': 'S3Prefix',
          'S3Uri': 's3://{}/{}/data/'.format(bucket, prefix),
        }
      }
    }
]
output_data_config = {'S3OutputPath': 's3://{}/{}/train_output'.format(bucket, prefix)}
optimizaton_metric_config = {'MetricName': 'AverageWeightedQuantileLoss'}
automl_problem_type_config ={
        'TimeSeriesForecastingJobConfig': {
            'ForecastFrequency': 'W',
            'ForecastHorizon': 4,
            'ForecastQuantiles': ['p50','p60','p70','p80','p90'],
            'Transformations': {
              'Filling': {
                  'unit_sales': {
                      'middlefill' : 'zero',
                      'backfill' : 'zero'
                      },
                  'promotion_email': {
                      'middlefill' : 'zero',
                      'backfill' : 'zero',
                      'futurefill' : 'zero'
                      },
                  'promotion_homepage': {
                      'middlefill' : 'zero',
                      'backfill' : 'zero',
                      'futurefill' : 'zero'
                      },
                  'promotion_email': {
                      'middlefill' : 'zero',
                      'backfill' : 'zero',
                      'futurefill' : 'zero'
                      },
                  'scaled_price': {
                      'middlefill' : 'value',
                      'middlefill_value' : '1',
                      'backfill' : 'value',
                      'backfill_value' : '1',
                      'futurefill' : 'value',
                      'futurefill_value' : '1'
                      }                                
              }
            },
            'TimeSeriesConfig': {
                'TargetAttributeName': 'unit_sales',
                'TimestampAttributeName': 'timestamp',
                'ItemIdentifierAttributeName': 'product_code',
                'GroupingAttributeNames': [
                    'location_code'
                ]
            }
        }
    }
print('AutoMLJobName:', auto_ml_job_name)
print('AutoMLJobInputDataConfig:', json.dumps(input_data_config))
print('OutputDataConfig:', json.dumps(output_data_config))
print('AutoMLProblemTypeConfig:', json.dumps(automl_problem_type_config))
# v2会失败因为当前不支持
sm.create_auto_ml_job_v2(
    AutoMLJobName=auto_ml_job_name,
    AutoMLJobInputDataConfig=input_data_config,
    OutputDataConfig=output_data_config,
    AutoMLProblemTypeConfig = automl_problem_type_config,
    AutoMLJobObjective=optimizaton_metric_config,
    RoleArn=role
)

AutoMLJobName: ts-20240807-094452
AutoMLJobInputDataConfig: [{"ChannelType": "training", "ContentType": "text/csv;header=present", "CompressionType": "None", "DataSource": {"S3DataSource": {"S3DataType": "S3Prefix", "S3Uri": "s3://sandbox-sagemaker/autopilot_time_series/data/"}}}]
OutputDataConfig: {"S3OutputPath": "s3://sandbox-sagemaker/autopilot_time_series/train_output"}
AutoMLProblemTypeConfig: {"TimeSeriesForecastingJobConfig": {"ForecastFrequency": "W", "ForecastHorizon": 4, "ForecastQuantiles": ["p50", "p60", "p70", "p80", "p90"], "Transformations": {"Filling": {"unit_sales": {"middlefill": "zero", "backfill": "zero"}, "promotion_email": {"middlefill": "zero", "backfill": "zero", "futurefill": "zero"}, "promotion_homepage": {"middlefill": "zero", "backfill": "zero", "futurefill": "zero"}, "scaled_price": {"middlefill": "value", "middlefill_value": "1", "backfill": "value", "backfill_value": "1", "futurefill": "value", "futurefill_value": "1"}}}, "TimeSeriesConfig": {"TargetAttr

ClientError: An error occurred (NotAuthorizedException) when calling the CreateAutoMLJobV2 operation: CreateAutoMLJobV2 API is currently not available for the requested region.

In [49]:
sm.create_auto_ml_job(
        AutoMLJobName=auto_ml_job_name,
        InputDataConfig=[
            {
                "DataSource": {
                    "S3DataSource": {
                        "S3DataType": "S3Prefix",
                        "S3Uri": 's3://{}/{}/data/'.format(bucket, prefix),
                    }
                },
                "TargetAttributeName": 'unit_sales',
            }
        ],
        OutputDataConfig={"S3OutputPath": 's3://{}/{}/train_output'.format(bucket, prefix)},
        ProblemType='Regression',
        AutoMLJobObjective={"MetricName": 'RMSE'},
        AutoMLJobConfig={
            "CompletionCriteria": {
                "MaxCandidates": 3,
            }
        },
        RoleArn=role,
    )

{'AutoMLJobArn': 'arn:aws-cn:sagemaker:cn-northwest-1:278103880173:automl-job/ts-20240807-094452',
 'ResponseMetadata': {'RequestId': 'b00c2fbb-624e-4b80-a1d3-59db6b9a9a35',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b00c2fbb-624e-4b80-a1d3-59db6b9a9a35',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '97',
   'date': 'Wed, 07 Aug 2024 09:45:21 GMT'},
  'RetryAttempts': 0}}

In [52]:
describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_run_status = describe_response["AutoMLJobStatus"]

while job_run_status not in ("Failed", "Completed", "Stopped"):
    describe_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    job_run_status = describe_response["AutoMLJobStatus"]

    print(
       datetime.datetime.now(), describe_response["AutoMLJobStatus"] + " - " + describe_response["AutoMLJobSecondaryStatus"]
    )
    sleep(60)

2024-08-07 18:04:27.919453 InProgress - ModelTuning
2024-08-07 18:05:28.181418 InProgress - MergingAutoMLTaskReports
2024-08-07 18:06:28.449782 InProgress - MergingAutoMLTaskReports
2024-08-07 18:07:28.725700 InProgress - MergingAutoMLTaskReports
2024-08-07 18:08:29.004358 InProgress - MergingAutoMLTaskReports
2024-08-07 18:09:29.238844 InProgress - MergingAutoMLTaskReports
2024-08-07 18:10:29.665221 InProgress - MergingAutoMLTaskReports
2024-08-07 18:11:30.186460 InProgress - MergingAutoMLTaskReports
2024-08-07 18:12:30.489103 InProgress - MergingAutoMLTaskReports
2024-08-07 18:13:30.797315 Completed - Completed


一旦训练完成, 就可以通过训练输出创建模型, 具体操作如下:

In [56]:
best_candidate = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['BestCandidate']
best_candidate_containers = best_candidate['InferenceContainers'] 
best_candidate_name = best_candidate['CandidateName']

print('BestCandidateName:',best_candidate_name)
print('BestCandidateContainers:',best_candidate_containers)
reponse = sm.create_model(
  ModelName = best_candidate_name,
  ExecutionRoleArn = role,
  Containers = best_candidate_containers
)

BestCandidateName: ts-20240807-094452v5E6DFA4YDBQ0K-003-d14cb079
BestCandidateContainers: [{'Image': '451049120500.dkr.ecr.cn-northwest-1.amazonaws.com.cn/sagemaker-sklearn-automl:2.5-1-cpu-py3', 'ModelDataUrl': 's3://sandbox-sagemaker/autopilot_time_series/train_output/ts-20240807-094452/data-processor-models/ts-20240807-094452-dpp1-1-f4188cb5af2c4ef0b97bbe68a5c881941bff9/output/model.tar.gz', 'Environment': {'AUTOML_SPARSE_ENCODE_RECORDIO_PROTOBUF': '1', 'AUTOML_TRANSFORM_MODE': 'feature-transform', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'application/x-recordio-protobuf', 'SAGEMAKER_PROGRAM': 'sagemaker_serve', 'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}}, {'Image': '451049120500.dkr.ecr.cn-northwest-1.amazonaws.com.cn/sagemaker-xgboost:1.3-1-cpu-py3', 'ModelDataUrl': 's3://sandbox-sagemaker/autopilot_time_series/train_output/ts-20240807-094452/tuning/ts-2024080-dpp1-xgb/ts-20240807-094452v5E6DFA4YDBQ0K-003-d14cb079/output/model.tar.gz', 'Environment': {'MAX_CONTENT_LENGTH':

In [57]:
endpoint_config_name = f"epc-{best_candidate_name}"
endpoint_name = f"ep-{best_candidate_name}"

production_variants = [
        {
            "InstanceType": "ml.m5.2xlarge",
            "InitialInstanceCount": 1,
            "ModelName": best_candidate_name,
            "VariantName": "AllTraffic",
        }
    ]
# 创建模型端点配置
epc_response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=production_variants
)

In [58]:
# 创建模型端点
sm.create_endpoint(
    EndpointName=endpoint_name, 
    EndpointConfigName=endpoint_config_name)

{'EndpointArn': 'arn:aws-cn:sagemaker:cn-northwest-1:278103880173:endpoint/ep-ts-20240807-094452v5E6DFA4YDBQ0K-003-d14cb079',
 'ResponseMetadata': {'RequestId': 'dc294cc6-9db1-4fe7-b753-a081407dbaf7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'dc294cc6-9db1-4fe7-b753-a081407dbaf7',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '124',
   'date': 'Wed, 07 Aug 2024 10:37:02 GMT'},
  'RetryAttempts': 0}}

In [59]:
# 检查模型端点ready状态
describe_response = sm.describe_endpoint(EndpointName=endpoint_name)

job_run_status = describe_response["EndpointStatus"]

while job_run_status not in ("Failed", "InService", "Stopped"):
    describe_response = sm.describe_endpoint(EndpointName=endpoint_name)
    job_run_status = describe_response["EndpointStatus"]

    print(
       datetime.datetime.now(), describe_response["EndpointStatus"])
    sleep(60)

2024-08-07 18:38:29.151752 Creating
2024-08-07 18:39:29.409315 Creating
2024-08-07 18:40:29.645523 Creating
2024-08-07 18:41:29.912827 InService


端点服务创建完毕就可以使用少了实时样例数据进行预测, 具体实现参考以下脚本
注意: 
- 样例数据构造一个[假的](https://amazon-forecast-samples.s3.amazonaws.com/autopilot/real-time-payload.csv), 真的数据可以来自业务
- 样例数据结构必须和训练的**数据数据结构一样**

In [67]:
# 读取实时数据
input_file = 'data/default-real-time-payload.csv'
f=open(input_file,'r')
inference_data = f.read()
f.close()

# 使用sagemaker-runtime进行调用
sm_client = boto3.client('sagemaker-runtime')
response = sm_client.invoke_endpoint(
    EndpointName= endpoint_name,
    Body= inference_data,
    ContentType = 'text/csv'
)
prediction = response['Body'].read().decode()

# 将结果写入文件
output_file = 'output/real-time-prediction-output.csv'
f=open(output_file,'w')
f.write(prediction)
f.close()

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from container-2 and could not load the entire response body. See https://cn-northwest-1.console.aws.amazon.com/cloudwatch/home?region=cn-northwest-1#logEventViewer:group=/aws/sagemaker/Endpoints/ep-ts-20240807-094452v5E6DFA4YDBQ0K-003-d14cb079 in account 278103880173 for more information.